# Introduction

All student names in group: Fia Hali, Andi Chen

I understand that my submission needs to be my own group's work: FH, AC

I understand that ChatGPT / Copilot / other AI tools are not allowed: FH, AC

Points: 10

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [ ]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 13.5 MB/s eta 0:00:00


In [ ]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(1 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(1 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(1 point)**

Solution:

1. number 5 occur at most once in the first row:
At most one of $X_{0,j}^5$ is true for j=range(9),  
~($X_{0,0}^5$ ⋀ $X_{0,1}^5$) ⋀ ~($X_{0,0}^5$ ⋀ $X_{0,2}^5$)... ~($X_{0,0}^5$ ⋀ $X_{0,8}^5$) ⋀     
~($X_{0,1}^5$ ⋀ $X_{0,2}^5$) ⋀ ~($X_{0,1}^5$ ⋀ $X_{0,3}^5$)... ~($X_{0,1}^5$ ⋀ $X_{0,8}^5$)⋀    
...     
~($X_{0,7}^5$ ⋀ $X_{0,8}^5$)

2. number 6 occur at most once in the first column:
At most one of $X_{i,0}^6$ is true for i=range(9),  
~($X_{0,0}^6$ ⋀ $X_{1,0}^6$) ⋀ ~($X_{0,0}^6$ ⋀ $X_{2,0}^6$)... ~($X_{0,0}^6$ ⋀ $X_{8,0}^6$) ⋀     
~($X_{1,0}^6$ ⋀ $X_{2,0}^6$) ⋀ ~($X_{1,0}^6$ ⋀ $X_{3,0}^6$)... ~($X_{1,0}^6$ ⋀ $X_{8,0}^6$)⋀    
...     
~($X_{7,0}^6$ ⋀ $X_{8,0}^6$)


3. number 9 occur at most once in the top left sub grid:
At most one of $X_{i,j}^9$ is true for i,j=range(3),  
~($X_{0,0}^9$ ⋀ $X_{0,1}^9$) ⋀ ~($X_{0,0}^9$ ⋀ $X_{0,2}^9$)⋀ ~($X_{0,1}^9$ ⋀ $X_{0,2}^9$) ⋀    
~($X_{1,0}^9$ ⋀ $X_{1,1}^9$) ⋀ ~($X_{1,0}^9$ ⋀ $X_{1,2}^9$)⋀ ~($X_{1,1}^9$ ⋀ $X_{1,2}^9$) ⋀      
~($X_{2,0}^9$ ⋀ $X_{2,1}^9$) ⋀ ~($X_{2,0}^9$ ⋀ $X_{2,2}^9$)⋀ ~($X_{2,1}^9$ ⋀ $X_{2,2}^9$)


## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(1 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [ ]:
from z3 import *

def solve_sudoku_bool(instance):
    solver = Solver()

    grid = [[[Bool("cell_%d_%d_%d" % (i, j, d+1)) for d in range(9)] for j in range(9)] for i in range(9)]

    # Each cell must contain exactly one number
    for i in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for d in range(9)], 1))

    # Each row must contain each number exactly once
    for d in range(9):
        for i in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for j in range(9)], 1))

    # Each column must contain each number exactly once
    for d in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for i in range(9)], 1))

    # Each 3x3 square must contain each number exactly once
    for d in range(9):
        for x in range(3):
            for y in range(3):
                solver.add(PbEq([(grid[3*x + i][3*y + j][d], 1) for i in range(3) for j in range(3)], 1))

    # Encoding the initial state of the Sudoku puzzle
    for i in range(9):
        for j in range(9):
            if instance[i][j] != 0:
                d = instance[i][j] - 1
                solver.add(grid[i][j][d])

    if solver.check() == sat:
        m = solver.model()
        r = [[0 for j in range(9)] for i in range(9)]
        for i in range(9):
            for j in range(9):
                for d in range(9):
                    if is_true(m[grid[i][j][d]]):
                        r[i][j] = d + 1
                        break
        print_matrix(r)
    else:
        print("Failed to solve")

instance = (
    (0, 1, 0, 4, 0, 2, 0, 5, 0),
    (5, 0, 0, 0, 0, 0, 0, 0, 6),
    (0, 0, 0, 3, 0, 1, 0, 0, 0),
    (7, 0, 5, 0, 0, 0, 4, 0, 8),
    (0, 0, 0, 0, 0, 0, 0, 0, 0),
    (2, 0, 8, 0, 0, 0, 5, 0, 9),
    (0, 0, 0, 9, 0, 6, 0, 0, 0),
    (6, 0, 0, 0, 0, 0, 0, 0, 2),
    (0, 7, 0, 1, 0, 3, 0, 4, 0)
)

solve_sudoku_bool(instance)


[[3, 1, 9, 4, 6, 2, 8, 5, 7],
 [5, 2, 4, 7, 9, 8, 1, 3, 6],
 [8, 6, 7, 3, 5, 1, 9, 2, 4],
 [7, 3, 5, 2, 1, 9, 4, 6, 8],
 [1, 9, 6, 8, 4, 5, 2, 7, 3],
 [2, 4, 8, 6, 3, 7, 5, 1, 9],
 [4, 5, 3, 9, 2, 6, 7, 8, 1],
 [6, 8, 1, 5, 7, 4, 3, 9, 2],
 [9, 7, 2, 1, 8, 3, 6, 4, 5]]


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(1 point)**

In [ ]:
from z3 import *

def check_solution_uniqueness(instance):
    solver = Solver()

    grid = [[[Bool(f"cell_{i}_{j}_{d+1}") for d in range(9)] for j in range(9)] for i in range(9)]

    # Each cell must hold exactly one value
    for i in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for d in range(9)], 1))

    # Each number must appear exactly once in each row, column, and 3x3 block
    for d in range(9):
        for i in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for j in range(9)], 1))
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for i in range(9)], 1))
        for x in range(3):
            for y in range(3):
                solver.add(PbEq([(grid[3*x + i][3*y + j][d], 1) for i in range(3) for j in range(3)], 1))

    # Apply the instance constraints
    for i in range(9):
        for j in range(9):
            if instance[i][j] != 0:
                d = instance[i][j] - 1
                solver.add(grid[i][j][d])


    assert solver.check() == sat
    model = solver.model()
    solution1 = [[0 for j in range(9)] for i in range(9)]
    for i in range(9):
        for j in range(9):
            for d in range(9):
                if is_true(model[grid[i][j][d]]):
                    solution1[i][j] = d + 1

    # Add a constraint to prevent this solution from occurring again
    solver.add(Or([Not(grid[i][j][solution1[i][j]-1]) for i in range(9) for j in range(9)]))


    if solver.check() == sat:
        print("A second solution exists, so the solution is not unique.")
    else:
        print("The solution is unique.")

    return solution1


instance = (
    (0, 1, 0, 4, 0, 2, 0, 5, 0),
    (5, 0, 0, 0, 0, 0, 0, 0, 6),
    (0, 0, 0, 3, 0, 1, 0, 0, 0),
    (7, 0, 5, 0, 0, 0, 4, 0, 8),
    (0, 0, 0, 0, 0, 0, 0, 0, 0),
    (2, 0, 8, 0, 0, 0, 5, 0, 9),
    (0, 0, 0, 9, 0, 6, 0, 0, 0),
    (6, 0, 0, 0, 0, 0, 0, 0, 2),
    (0, 7, 0, 1, 0, 3, 0, 4, 0)
)

unique_solution = check_solution_uniqueness(instance)
print_matrix(unique_solution)


The solution is unique.
[[3, 1, 9, 4, 6, 2, 8, 5, 7],
 [5, 2, 4, 7, 9, 8, 1, 3, 6],
 [8, 6, 7, 3, 5, 1, 9, 2, 4],
 [7, 3, 5, 2, 1, 9, 4, 6, 8],
 [1, 9, 6, 8, 4, 5, 2, 7, 3],
 [2, 4, 8, 6, 3, 7, 5, 1, 9],
 [4, 5, 3, 9, 2, 6, 7, 8, 1],
 [6, 8, 1, 5, 7, 4, 3, 9, 2],
 [9, 7, 2, 1, 8, 3, 6, 4, 5]]


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be less than 5000. **(1 point)**

In [ ]:
## It would be 200 solutions
from z3 import *

def solve_sudoku_bool(instance):
    solver = Solver()

    grid = [[[Bool("cell_%d_%d_%d" % (i, j, d+1)) for d in range(9)] for j in range(9)] for i in range(9)]

    # Each cell must contain exactly one number
    for i in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for d in range(9)], 1))

    #  Each row must contain each number exactly once
    for d in range(9):
        for i in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for j in range(9)], 1))

    #  Each column must contain each number exactly once
    for d in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for i in range(9)], 1))

    #  Each 3x3 square must contain each number exactly once
    for d in range(9):
        for x in range(3):
            for y in range(3):
                solver.add(PbEq([(grid[3*x + i][3*y + j][d], 1) for i in range(3) for j in range(3)], 1))

    # Encoding the initial state of the Sudoku puzzle
    for i in range(9):
        for j in range(9):
            if instance[i][j] != 0:
                d = instance[i][j] - 1  # Adjusting index for 0-based indexing
                solver.add(grid[i][j][d])


    solution_count = 0
    solutions = []
    while solution_count < 5000 and solver.check() == sat:
        model = solver.model()
        current_solution = [[0 for j in range(9)] for i in range(9)]
        # Extracting the solution from the model
        for i in range(9):
            for j in range(9):
                for d in range(9):
                    if is_true(model[grid[i][j][d]]):
                        current_solution[i][j] = d + 1
                        break
        solutions.append(current_solution)


        new_constraints = [grid[i][j][current_solution[i][j] - 1] != model[grid[i][j][current_solution[i][j] - 1]]
                        for i in range(9) for j in range(9)]
        solver.add(Or(new_constraints))

        solution_count += 1

    return solution_count


instance = (
    (0, 0, 0, 4, 0, 2, 0, 5, 0),
    (5, 0, 0, 0, 0, 0, 0, 0, 6),
    (0, 0, 0, 3, 0, 1, 0, 0, 0),
    (7, 0, 5, 0, 0, 0, 4, 0, 8),
    (0, 0, 0, 0, 0, 0, 0, 0, 0),
    (2, 0, 8, 0, 0, 0, 5, 0, 9),
    (0, 0, 0, 9, 0, 6, 0, 0, 0),
    (6, 0, 0, 0, 0, 0, 0, 0, 2),
    (0, 7, 0, 1, 0, 3, 0, 4, 0)
)

count = solve_sudoku_bool(instance)
print(f"Number of unique solutions: {count}")

Number of unique solutions: 200


# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(2 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [ ]:
from z3 import *

def solve_sudoku_miracle(instance):
    solver = Solver()

    grid = [[[Bool("cell_%d_%d_%d" % (i, j, d+1)) for d in range(9)] for j in range(9)] for i in range(9)]

    # Each cell must contain exactly one number
    for i in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for d in range(9)], 1))

    #  Each row must contain each number exactly once
    for d in range(9):
        for i in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for j in range(9)], 1))

    #  Each column must contain each number exactly once
    for d in range(9):
        for j in range(9):
            solver.add(PbEq([(grid[i][j][d], 1) for i in range(9)], 1))

    #  Each 3x3 square must contain each number exactly once
    for d in range(9):
        for x in range(3):
            for y in range(3):
                solver.add(PbEq([(grid[3*x + i][3*y + j][d], 1) for i in range(3) for j in range(3)], 1))

    for i in range(9):
        for j in range(9):
            for k in range(9):
                # Knight's move constraint
                for dx, dy in [(-2, -1), (-2, 1), (-1, -2), (-1, 2),
                               (1, -2), (1, 2), (2, -1), (2, 1)]:
                    if 0 <= i + dx < 9 and 0 <= j + dy < 9:
                      solver.add(PbLe([((grid[i][j][k]), 1), ((grid[i + dx][j + dy][k]), 1)], 1))  # Knight move constraint

                # # King's move constraint
                for a in range(-1,2):
                  for b in range(-1,2):
                    if abs(a)+abs(b)==2 and i+a<9 and i+a>=0 and j+b<9 and j+b>=0:
                      solver.add(PbLe([(grid[i][j][d], 1), (grid[i + a][j + b][d], 1)], 1))

                # Consecutive numbers constraint
                if k < 8:
                    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                        if 0 <= i + dx < 9 and 0 <= j + dy < 9:
                          solver.add(PbLe([(grid[i][j][k], 1), (grid[i + dx][j + dy][k - 1], 1), (grid[i + dx][j + dy][k + 1], 1)], 1))

    # Encoding the initial state of the Sudoku puzzle
    for i in range(9):
        for j in range(9):
            if instance[i][j] != 0:
                d = instance[i][j] - 1  # Adjusting index for 0-based indexing
                solver.add(grid[i][j][d])

    if solver.check() == sat:
        m = solver.model()
        r = [[0 for j in range(9)] for i in range(9)]
        for i in range(9):
            for j in range(9):
                for d in range(9):
                    if is_true(m[grid[i][j][d]]):
                        r[i][j] = d + 1
                        break
        print_matrix(r)
    else:
        print("Failed to solve")

instance =((0, 0, 0, 0, 0, 0, 0, 0, 0),
            (0, 0, 0, 0, 0, 0, 0, 0, 0),
            (0, 0, 0, 0, 0, 0, 0, 0, 0),
            (0, 0, 0, 0, 0, 0, 0, 0, 0),
            (0, 0, 1, 0, 0, 0, 0, 0, 0),
            (0, 0, 0, 0, 0, 0, 2, 0, 0),
            (0, 0, 0, 0, 0, 0, 0, 0, 0),
            (0, 0, 0, 0, 0, 0, 0, 0, 0),
            (0, 0, 0, 0, 0, 0, 0, 0, 0))

solve_sudoku_miracle(instance)

[[4, 8, 3, 7, 2, 6, 1, 5, 9],
 [7, 2, 6, 1, 5, 9, 4, 8, 3],
 [1, 5, 9, 4, 8, 3, 7, 2, 6],
 [8, 3, 7, 2, 6, 1, 5, 9, 4],
 [2, 6, 1, 5, 9, 4, 8, 3, 7],
 [5, 9, 4, 8, 3, 7, 2, 6, 1],
 [3, 7, 2, 6, 1, 5, 9, 4, 8],
 [6, 1, 5, 9, 4, 8, 3, 7, 2],
 [9, 4, 8, 3, 7, 2, 6, 1, 5]]




---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(1 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$
 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$

(add markdown tables here)

| p    |  q |   r| s     |  t   | ~s|(p ->q) ->r| s -> ~p | t->q| r|
|--- -| ----|-----|------|------|------|------|------|------|------|
|T    | T   | T   |F     |**T** |**T** |**T** |**T** |**T** |**T**
| T   | T   | T   |F     |F     |T     |T     |T     |T     |T
| T   | T   | F   |T     |T     |F     |F     |F     |T     |F
| T   | T   | F   |T     |F     |F     |F     |F     |T     |F
| T   | F   | T   |F     |T     |T     |T     |T     |F     |T
| T   | F   | T   |F     |F     |T     |T     |T     |T     |T
| T   | F   | F   |T     |T     |F     |F     |F     |F     |F
| T   | F   | F   |T     |F     |F     |F     |F     |F     |F
| F   | T   | T   |F     |**T** |**T** |**T** |**T** |**T** |**T**
| F   | T   | T   |F     |F     |T     |T     |T     |T     |T
| F   | T   | F   |T     |T     |F     |F     |T     |T     |F
| F   | T   | F   |T     |F     |F     |F     |T     |T     |F
| F   | F   | T   |F     |**T** |**T** |**T** |**T** |**T** |**T**
| F   | F   | T   |F     |F     |T     |T     |T     |T     |T
| F   | F   | F   |T     |T     |F     |F     |T     |T     |F
| F   | F   | F   |T     |F     |F     |F     |T     |T     |F

row 1,9,and 13 are true, therefore its true


| p    |  q |   q→(p→(p→(q→p)))|
|--- - |----|-----|
|T    | T   | **T**   |
| T   | F   | **T**   |
| F   | T   | **T**   |
| F   | F   | **T**   |

entailment is true in all case, therefore its true




---


 ## Logical Entailment using z3py

 Use Z3py to prove the above three entailments are true using satisfiability. See section 7.5 in the book for details. **(1 point)**

In [ ]:
# Q1
p, q, r, s, t = Bools('p q r s t')

# the 5 expressions
ex1 = Implies(Implies(p, q), r)
ex2 = Implies(s, Not(p))
ex3 = t
ex4 = Not(s)
ex5 = Implies(t, q)
conclusion = r
# solve
s = Solver()
s.add((And(ex1, ex2, ex3, ex4, ex5, Not(conclusion))))

if s.check() == unsat:
    print("Q1 is true.")
else:
    print("Q1 is not true.")

#Q2
ex1 = Implies(q, Implies(p, Implies(p, Implies(q, p))))
# solve
s = Solver()
s.add(Not(Implies(BoolVal(True), ex1)))

if s.check() == unsat:
    print("Q2 is true.")
else:
    print("Q2 is not true.")



Q1 is true.
Q2 is true.
